Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
import numpy as np
from copy import deepcopy

In [42]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [43]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS) ## valori degli oggetti
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) ## pesi degli oggetti (n.b ogni peso ha più dimensioni e i constraint devono essere rispettati in ogni dimensione)
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) ## tante quante le dimensioni 

In [5]:
CONSTRAINTS
# ogni zaino è una riga e ogni colonna è il constraint sulla dimensione

array([[379, 157],
       [371, 437]], dtype=int32)

In [20]:
# A random solution
solution = np.array(
    [np.random.random(NUM_ITEMS) < 0.5 for _ in range(NUM_KNAPSACKS)], dtype=np.bool
)

In [45]:
def tweak_non_valida(knapsacs:  list[set] ) -> list[set]:
    ## conviene togliere un oggetto da uno zaino che è in sovrappeso
    new_bags = deepcopy(knapsacs)
    # trova uno zaino in sovrappeso
    overweight_bags = []
    for b in range(NUM_KNAPSACKS):
        total_weight = np.zeros(NUM_DIMENSIONS)
        for i in range(NUM_ITEMS):
            if new_bags[b][i]:
                total_weight += WEIGHTS[i]
        if np.any(total_weight > CONSTRAINTS[b]):
            overweight_bags.append(b)
    for b in overweight_bags:
        if len(new_bags[b]) > 0:
            item_to_remove = np.random.choice(list(new_bags[b][new_bags[b]>0]))
            new_bags[b][item_to_remove] = False
            break  # tweak only one bag
    return new_bags

In [34]:
def tweak_valida(knapsack: list[set]) -> list[set]:
    ## conviene aggiungere un oggetto a uno zaino che non è in sovrappeso
    new_bags = deepcopy(knapsack)
    for b in range(NUM_KNAPSACKS):

        # calcolo il peso attuale dello zaino
        knapsack_weight = np.zeros(NUM_DIMENSIONS)
        for i in range(NUM_ITEMS):
            if new_bags[b][i]:
                knapsack_weight += WEIGHTS[i]
        
        ## prendo un oggetto a caso se non è presente in nessuno zaino lo aggiungo a quello più vuoto al contrario lo scambio
        item_to_add = np.random.choice([i for i in range(NUM_ITEMS) ])
        if item_to_add in new_bags[b]:
            # lo tolgo e lo metto in un altro zaino
            new_bags[b][item_to_add]=False
            other_bags = [i for i in range(NUM_KNAPSACKS) if i != b]
            other_bag = np.random.choice(other_bags)
            new_bags[other_bag][item_to_add] = True
        else:
            ## lo aggiungo e lo tolgo da un altro zaino se presente

            new_knapsack_weight = knapsack_weight+ WEIGHTS[item_to_add]
            if np.any(new_knapsack_weight > CONSTRAINTS[b]):
                # non posso aggiungerlo
                continue
            new_bags[b][item_to_add]=False
            for ob in range(NUM_KNAPSACKS):
                if ob != b and item_to_add in new_bags[ob]:
                    new_bags[ob][item_to_add] = False
            
    return new_bags

In [28]:
## il costo di una soluzione è dato dalla somma dei valori degli oggetti meno la somma dei sovrappesi
# se la soluzione è valida il costo è negativo
def cost(solution: list[set]) -> (int, int):
    total_value = 0
    total_overweight = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS))
    for b in range(NUM_KNAPSACKS): 
        knapsac_weight=np.zeros(NUM_DIMENSIONS)
        knapsac_value=0
        for i in range(NUM_ITEMS):
            if solution[b][i]:
                knapsac_value += VALUES[i]
                knapsac_weight+=WEIGHTS[i]

        total_value+= knapsac_value
        overweight= knapsac_weight-CONSTRAINTS[b]
        total_overweight[b]=overweight
        ## costo deve essere negativo se eccede di peso e positivo se no
    cost=0
    ## somma de pesi che eccedonono -> soluzione non valida
    cost = np.sum(total_overweight[total_overweight > 0])
    
    ## se la soluzione è valida 
    if cost==0:
        cost=np.sum(total_overweight) ## sarà negativo o zero

    return (cost, total_value)

In [25]:
solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
for item in range(NUM_ITEMS):
    if np.random.random() < 0.5:  # probabilità di assegnare l'oggetto
        knapsack = np.random.randint(NUM_KNAPSACKS)
        solution[knapsack, item] = True
cost(solution)

(np.float64(18.0), np.int32(333))

In [ ]:
# current_solution =solution = np.array(
#     [np.random.random(NUM_ITEMS) < 0.5 for _ in range(NUM_KNAPSACKS)], dtype=np.bool
# )

# Ogni oggetto viene assegnato a uno zaino casuale (o nessuno)
current_solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
for item in range(NUM_ITEMS):
    if np.random.random() < 0.5:  # probabilità di assegnare l'oggetto
        knapsack = np.random.randint(NUM_KNAPSACKS)
        current_solution[knapsack, item] = True

(current_cost, current_value) = cost(current_solution)


MAX_STEPS = 500


for steps in range(MAX_STEPS):
    if current_cost == 0:
        break

    if current_cost < 0:
        ## soluzione valida
        new_solution = tweak_valida(current_solution)
        (new_cost, new_value) = cost(new_solution)
        if new_cost > 0:
            ## la nuova soluzione non è valida -> scarto
            continue
        if new_cost > current_cost and new_value> current_value: # current < new <= 0
            ## la nuova soluzione è valida e il costo si avvicina a zero sempre di più (costo ideale)
            current_cost= new_cost
            current_solution = new_solution
            current_value = new_value
            continue

    else: 
        ## soluzione non valida
        new_solution = tweak_non_valida(current_solution)
        (new_cost, new_value) = cost(new_solution)

        if new_cost > 0: 
        ## la nuova soluzione non è valida anche essa 
            if new_cost < current_cost: # 0<= new < current
                # la nuova soluzione si avvicina a essere valida 
                current_cost = new_cost
                current_solution = new_solution
                continue
            else:
                continue
        else:
            ## la nuova soluzione è valida 
            current_cost = new_cost
            current_solution = new_solution
            current_value = new_value
            continue
    

print(current_cost, current_value)

-479618.0 2840


In [63]:
solution= current_solution
solution

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False],
       [ True,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, Fals

In [64]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

np.False_

In [65]:
# Check if the solution is valid
all_knapsacks = np.any(solution, axis=0)
np.all(WEIGHTS[all_knapsacks].sum(axis=0) < CONSTRAINTS)

np.False_

## TEST PROBLEMS

In [51]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = np.random.randint(0, 100, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 


In [60]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = np.random.randint(0, 1000, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(0, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 


In [57]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = np.random.randint(0, 1000, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 

